In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1_Tr_J/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
19846,70.441084,0.371688,14.930385,0.895400,1.830435,0.057119,0.866247,1.743451,4.468076,4.396114
19847,70.445469,0.371500,14.929297,0.895397,1.829782,0.057101,0.865378,1.741588,4.468006,4.396482
19848,70.450608,0.371279,14.928022,0.895394,1.829017,0.057079,0.864360,1.739406,4.467923,4.396912
19849,70.456460,0.371028,14.926567,0.895390,1.828146,0.057054,0.863201,1.736923,4.467828,4.397403
20334,69.486228,0.412294,15.144012,0.895242,1.968498,0.061023,1.064592,2.171610,4.478991,4.317511


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
19846,70.441084,0.371688,14.930385,0.895400,1.830435,0.057119,0.866247,1.743451,4.468076,4.396114
19847,70.445469,0.371500,14.929297,0.895397,1.829782,0.057101,0.865378,1.741588,4.468006,4.396482
19848,70.450608,0.371279,14.928022,0.895394,1.829017,0.057079,0.864360,1.739406,4.467923,4.396912
19849,70.456460,0.371028,14.926567,0.895390,1.828146,0.057054,0.863201,1.736923,4.467828,4.397403
20334,69.486228,0.412294,15.144012,0.895242,1.968498,0.061023,1.064592,2.171610,4.478991,4.317511
...,...,...,...,...,...,...,...,...,...,...
208816,73.434814,0.242304,13.923243,0.884466,1.350652,0.043278,0.374717,0.714728,4.367559,4.664239
208817,73.450299,0.241647,13.916517,0.884356,1.348033,0.043201,0.372749,0.710736,4.366730,4.665733
208818,73.464622,0.241041,13.910280,0.884253,1.345609,0.043130,0.370934,0.707055,4.365959,4.667117
208819,73.477805,0.240482,13.904527,0.884158,1.343378,0.043065,0.369268,0.703679,4.365247,4.668391


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:90000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [14]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1_Tr_J/area1_Tr_J_2_results_excel.txt")

In [15]:
df_final = pd.concat([df_final1, df_final2])

In [16]:
df_final

,Q,P,K
SAMPLE,,,
19846,21.839468,41.724163,29.757681
19847,21.847382,41.719882,29.759805
19848,21.856672,41.714834,29.762271
19849,21.867210,41.709151,29.765091
20334,20.113865,42.573269,29.302453
...,...,...,...
208816,27.435339,36.804602,31.257043
208817,27.463169,36.778487,31.265145
208818,27.488833,36.754368,31.272709


## QAPF classification

In [17]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [18]:
df_final["QAPF"].unique()

array(['monzo granite', 'quartz monzonite'], dtype=object)

In [19]:
df_final.to_excel("../_CIPW/CIPW/AREA1_Tr_J/qapf.xlsx", index=True)

-------

In [20]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1_Tr_J/QAPF_counts.xlsx")

-----

In [21]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [22]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control
QAPF_control.to_excel("../_CIPW/CIPW/AREA1_Tr_J/QAPF_control_points.xlsx", index=True)